In [1]:
import pandas as pd
import glob
import datetime

# Define the string that the file names must start with
file_filter = 'AAPL'

# Define the path to the directory containing the CSV files
path = '/Users/chrisjackson/OptionsApp/data/' + file_filter

# Use glob to get a list of all CSV files in the directory that start with the specified string
all_files = glob.glob(path + "/" + file_filter + "*.csv")

def add_sample_date(file_name):
    # Read the CSV file into a data frame
    df = pd.read_csv(file_name)
    # Add a column to the data frame containing the date of the sample
    df['sample_date'] = file_name[-16:-6]
    return df

# Combine all CSV files into a single data frame
df = pd.concat((add_sample_date(f) for f in all_files), ignore_index=True)

# Define a function to extract the expiration date from an OCC contract symbol
def extract_expiry_date(occ_symbol):
    year = int(occ_symbol[4:6])
    month = int(occ_symbol[6:8])
    day = int(occ_symbol[8:10])
    expiry_date = datetime.datetime(year + 2000, month, day)
    return expiry_date

# Define a function to convert the timestamp to a datetime object
def convert_timestamp(timestamp):
    if len(timestamp) > 8:
        timestamp = timestamp[-8:]        
    hour = int(timestamp[0:2])
    minute = int(timestamp[3:5])
    second = int(timestamp[6:8])
    return datetime.time(hour, minute, second)

# Add a new column to the DataFrame with the expiry date
df['expiryDate'] = df['contractSymbol'].apply(extract_expiry_date)

# Add a new column with days to expiry
df['daysToExpiry'] = df['expiryDate'] - df['sample_date'].apply(pd.to_datetime)

# Convert the 'lastTradeDate' and 'timestamp' columns to datetime objects for future calculations
df['lastTradeDate'] = pd.to_datetime(df['lastTradeDate'])

# df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S')
df['timestamp'] = df['timestamp'].apply(convert_timestamp)

# Drop the currency column
df.drop('currency', axis=1, inplace=True)
df.info()


<string>:1: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.


In [ ]:
# Need to sort this into a multi-index dataframe by sample date and contract symbol

# Set the multi-index
df.set_index(['sample_date', 'contractSymbol'], inplace=True)

# Sort by the timestamp
df = df.sort_values(by=['timestamp', 'expiryDate', 'strike'])


In [ ]:
df.index.levels

In [ ]:
df.loc['2023-04-19']

In [ ]:
#Retreive the timestamp as a datetime object
def get_timestamp_breakdown(timestamp):
    hour = int(timestamp[0:2])
    minute = int(timestamp[3:5])
    second = int(timestamp[6:8])
    return (hour, minute, second)

In [ ]:
# Get the value where the in the money column changes from True to False, pass in the first timestamp level of the dataframe only for now
def in_the_money(df):
    # Add a column to the data frame containing the previous value of the inTheMoney column
    df['previous_value'] = df['inTheMoney'].shift(1)
    df['change_to_false'] = (df['previous_value'] == True) & (df['inTheMoney'] == False)

    # Get the index of the first row where the value changed to False
    first_change_index = df['change_to_false'].idxmax()

    # Get the first row where the value changed to False
    first_change_row = df.loc[first_change_index]

    return first_change_row


In [ ]:
import matplotlib.pyplot as plt

# Data exploration and processing

# Find where the in the money column changes from False to True, might be useful for plotting or modelling
in_the_money_strike_at_open = []
for date in df.index.levels[0]:
    tempdf = df.loc[date]
    fist_timestamp = tempdf['timestamp'].min()
    first_change_row = in_the_money(tempdf[tempdf['timestamp'] == fist_timestamp])
    in_the_money_strike_at_open.append(first_change_row)

# Select the ten options adjacent to the strike price at the open
for x in in_the_money_strike_at_open:
    print(x)

 
# Calculate the 90th percentile of volume for each day
volume_threshold = df.groupby(level=0)['volume'].quantile(0.9)

# For each day, select the options with volume greater than or equal to the threshold
top_volume_options = df[df.groupby('sample_date')['volume'].transform(lambda x: x >= volume_threshold[x.name])]

# Sort the DataFrame by sample_date and timestamp in descending order
top_volume_options.sort_values(['sample_date', 'timestamp'], ascending=[True, False], inplace=True)

# Group the options by their contract symbol, and calculate the average and standard deviation of the price changes
grouped = top_volume_options.groupby('contractSymbol')
mean_changes = grouped['change'].mean()
std_changes = grouped['change'].std()

# Flag any options whose price change is more than two standard deviations away from the mean
threshold = 2 * std_changes
flagged = (df['change'] - mean_changes[df['expiryDate']].values) > threshold[df['expiryDate']].values

# Create a scatter plot of the price changes, highlighting the flagged options
fig, ax = plt.subplots()
ax.scatter(df['expiryDate'], df['change'], alpha=0.5)
ax.scatter(df.loc[flagged, 'expiryDate'], df.loc[flagged, 'change'], c='red')
ax.set_xlabel('Expiration Date')
ax.set_ylabel('Price Change')
ax.set_title('Flagged Options')
plt.show()


In [ ]:
# Group the options by their expiration date, and calculate the mean volume for each expiration date
grouped = df.groupby('expiryDate')['volume'].mean()

# Sort the expiration dates by volume in descending order, and select the top 3 expiration dates
top3 = grouped.sort_values(ascending=False).head(3)

# Filter the data frame to only include options with the top 3 expiration dates
df_top3 = df[df['expiryDate'].isin(top3.index)]

# Group the top 3 expiration dates by their expiration date, and calculate the average and standard deviation of the price changes
grouped_top3 = df_top3.groupby('expiryDate')
mean_changes = grouped_top3['change'].mean()
std_changes = grouped_top3['change'].std()

# Flag any options whose price change is more than two standard deviations away from the mean
threshold = 2 * std_changes
flagged = (df_top3['change'] - mean_changes[df_top3['expiryDate']].values) > threshold[df_top3['expiryDate']].values

# Create a scatter plot of the price changes for the top 3 expiration dates, highlighting the flagged options
fig, ax = plt.subplots(figsize=(15, 7.5))
for i, (name, group) in enumerate(df_top3.groupby('expiryDate')):
    ax.scatter(group['timestamp'], group['change'], alpha=0.5, label=name if i == 0 else None)
ax.scatter(df_top3.loc[flagged, 'timestamp'], df_top3.loc[flagged, 'change'], c='red')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Price Change')
ax.set_title('Flagged Options (Top 3 Expiration Dates by Volume)')
ax.legend()
plt.show()
